# Demo notebook : SelfPlay with ONNX models

## Setup

In [895]:
MODE = "local"  # "colab" | "colab-dev" | "local"

In [896]:
if MODE == "colab":
    !pip install -q lczerolens
elif MODE == "colab-dev":
    !rm -r lczerolens
    !git clone https://github.com/Xmaster6y/lczerolens -b main
    !pip install -q ./lczerolens

## Download ONNX Models

In [897]:
!gdown 1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO -O maia1100.onnx
!gdown 1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl -O maia1900.onnx

Downloading...
From: https://drive.google.com/uc?id=1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1100.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 32.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1900.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 34.6MB/s]


## Import Libraries

In [898]:
# ruff: noqa: E402
import importlib
import lczerolens.sampling as sampling
import lczerolens.search as search
import numpy as np

for module in (sampling, search):
    importlib.reload(module)

import IPython.display
from lczerolens import LczeroBoard, LczeroModel
from lczerolens.sampling import MCTSSampler, ModelSampler, SelfPlay
from lczerolens.search import MaterialHeuristic, MCTS, Node

# 1. SelfPlay (MCTS + NN) Maia1100 VS Maia1900

## Load & Setup Model Samplers

In [899]:
maia_1100_model = LczeroModel.from_path("maia1100.onnx")
maia_1900_model = LczeroModel.from_path("maia1900.onnx")
white_sampler = MCTSSampler(model=maia_1100_model, num_simulations=100, use_argmax=True)
black_sampler = ModelSampler(model=maia_1900_model, use_argmax=True)

## Initialize SelfPlay

In [900]:
self_play = SelfPlay(white=white_sampler, black=black_sampler)
logs = []


def report_fn(log, to_play):
    logs.append((log, to_play))

## Play Game

In [901]:
board = LczeroBoard()

game_moves, final_board = self_play.play(board=board, max_moves=10, report_fn=report_fn)

## Display Results

In [902]:
print("Game moves:", game_moves)
print("Game result:", final_board.result())

Game moves: [Move.from_uci('e2e4'), Move.from_uci('c7c5'), Move.from_uci('g1f3'), Move.from_uci('d7d6'), Move.from_uci('f1c4'), Move.from_uci('e7e6'), Move.from_uci('b1c3'), Move.from_uci('g8f6'), Move.from_uci('d2d3'), Move.from_uci('f8e7')]
Game result: *


# 2. Test heuristic : Material advantage

In [903]:
fen = "6nr/pp3p1p/k1p5/8/1QN5/2P1P3/4KPqP/8 b - - 5 26"
iterations = 100
c_puct = 1.0
n_parallel_rollouts = 1

In [904]:
board = LczeroBoard(fen)
heuristic = MaterialHeuristic()
mcts = MCTS(c_puct=c_puct, n_parallel_rollouts=n_parallel_rollouts)
root = Node(board, parent=None)

In [905]:
mcts.search_(root, heuristic=heuristic, iterations=iterations)
best_move_idx = np.argmax(root.visits)
best_move = root.legal_moves[best_move_idx].uci()
print("Best move :", best_move)

Best move : g8h6


In [906]:
graph = MCTS.render_tree(root, max_depth=30)
display(IPython.display.HTML(graph))

# 3. SelfPlay : Choose both heuristic to make play one against another

## Load & Setup Model Samplers

In [907]:
# white_sampler = RandomSampler()
# white_sampler = MCTSSampler(model=None, _heuristic=MaterialHeuristic(), use_argmax=False)
# white_sampler = MCTSSampler(model=None, _heuristic=DummyHeuristic(), use_argmax=False)
# white_sampler = ModelSampler(model=maia_1100_model, use_argmax=False)
white_sampler = ModelSampler(model=maia_1900_model, use_argmax=False)

# black_sampler = RandomSampler()
# black_sampler = MCTSSampler(model=None, _heuristic=DummyHeuristic(), use_argmax=False)
# black_sampler = MCTSSampler(model=None, _heuristic=MaterialHeuristic(), use_argmax=False)
# black_sampler = ModelSampler(model=maia_1100_model, use_argmax=False)
black_sampler = ModelSampler(model=maia_1900_model, use_argmax=False)

## Initialize SelfPlay

In [908]:
self_play = SelfPlay(white=white_sampler, black=black_sampler)
logs = []

## Play Game

In [909]:
board = LczeroBoard()

game_moves, final_board = self_play.play(board=board, max_moves=100, report_fn=report_fn)

## Display Results

In [910]:
print("Game moves:", game_moves)
print("Game result:", final_board.result())

Game moves: [Move.from_uci('e2e4'), Move.from_uci('d7d6'), Move.from_uci('d2d4'), Move.from_uci('g8f6'), Move.from_uci('b1c3'), Move.from_uci('g7g6'), Move.from_uci('f1d3'), Move.from_uci('f8g7'), Move.from_uci('f2f4'), Move.from_uci('e7e6'), Move.from_uci('g1f3'), Move.from_uci('b8d7'), Move.from_uci('e4e5'), Move.from_uci('f6d5'), Move.from_uci('c3e4'), Move.from_uci('d5f4'), Move.from_uci('c1f4'), Move.from_uci('d6e5'), Move.from_uci('d4e5'), Move.from_uci('h7h6'), Move.from_uci('h2h4'), Move.from_uci('b7b6'), Move.from_uci('d1d2'), Move.from_uci('c8b7'), Move.from_uci('g2g4'), Move.from_uci('f7f5'), Move.from_uci('e5f6'), Move.from_uci('d7f6'), Move.from_uci('e4f6'), Move.from_uci('d8f6'), Move.from_uci('h1f1'), Move.from_uci('f6b2'), Move.from_uci('d3g6'), Move.from_uci('e8e7'), Move.from_uci('f4c7'), Move.from_uci('b2a1'), Move.from_uci('e1e2'), Move.from_uci('a1e5'), Move.from_uci('f3e5'), Move.from_uci('g7e5'), Move.from_uci('c7e5'), Move.from_uci('b7a6'), Move.from_uci('e2e1')

# Results (1 round) :

Random * Random

Random 0-1 maia1900

Random 0-1 Dummy

Random * Material

Random 0-1 maia1100

---

Dummy * Material

Dummy 0-1 Material

Dummy * Maia1100

Dummy 0-1 Maia1900

---

Material * Material

Material * Maia1100

Material * Maia1900

---

Maia1100 1-0 Maia1100

Maia1100 * Maia1900

---

Maia1900 1-0 Maia1900.